## 1. Продвинутые методы Pandas
<a class="anchor" id="0"></a>
### Содержание:
* [1. Введение](#tt1)
* [2. Сортировка данных в DF](#tt2)
* [3. Группировка данных в DF](#tt3)
* [3.1 Метод `groupby()`](#tt3-1)  
* [3.2 Группировка с одной агрегацией](#tt3-2)  
* [3.2 Группировка с несколькими агрегациями](#tt3-3)  
* [3.2 Практика по группировкам данных](#tt3-4)  
* [4. Сводные таблицы](#tt4)
* [4.1 Метод groupby() для сводных таблиц + метод unstack()](#tt4-1)
* [4.2 Метод pivot_table()](#tt4-2)
* [4.3 Многомерные сводные таблицы](#tt4-3)
* [4.4 Доступ к данным в сводной таблице](#tt4-4)
* [4.5 Практика по сводным таблицам](#tt4-5)
* [5. Объединение данных в DF: знакомство с новым](#tt5)
* [6. Объединение данных в DF: concat](#tt6)
* [7. Объединение данных в DF: join, merge](#tt7)
* [7.1 Метод join()](#tt7-1)
* [7.2 Метод merge()](#tt7-2)
* [8. Практика. Закрепление знаний](#tt8)


## Введение <a class="anchor" id="tt1"></a>

**ВАЖНО!** Выявлять и устанавливать взаимосвязи между данными (признаками) и целевым признаком, так же хорошо бы уметь сортировать

**ТОЖЕ ВАЖНО** 1 большая таблица лучше 5ти маленьких. Объединяй.

In [1]:
import pandas as pd
melb_data = pd.read_csv('melb_data_fe.csv', sep=',')
melb_df = melb_data.copy()
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Suburb           13580 non-null  object 
 1   Rooms            13580 non-null  int64  
 2   Type             13580 non-null  object 
 3   Price            13580 non-null  float64
 4   Method           13580 non-null  object 
 5   SellerG          13580 non-null  object 
 6   Date             13580 non-null  object 
 7   Distance         13580 non-null  float64
 8   Postcode         13580 non-null  int64  
 9   Bedroom          13580 non-null  int64  
 10  Bathroom         13580 non-null  int64  
 11  Car              13580 non-null  int64  
 12  Landsize         13580 non-null  float64
 13  BuildingArea     13580 non-null  float64
 14  CouncilArea      12211 non-null  object 
 15  Lattitude        13580 non-null  float64
 16  Longtitude       13580 non-null  float64
 17  Regionname  

`csv`-файл не хранит в себе информацию о типах данных столбцов, поэтому при чтении `Pandas` **автоматически определяет тип данных столбца**. Не забывайте об этом, обмениваясь преобразованными данными с вашими коллегами.

In [2]:
# преобразуем столбец Date в формат datetame
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# выделим в отдельную переменную квартал продаж
quarter = melb_df['Date'].dt.quarter
# смотрим информацию о значениях квартальных продаж
quarter.value_counts()

3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64

In [3]:
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,Method,5,object
3,WeekdaySale,7,int64
4,Regionname,8,object
5,Rooms,9,int64
6,Bathroom,9,int64
7,Car,11,int64
8,StreetType,11,object
9,MonthSale,12,int64


In [4]:
# Столбцы "исключения"
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
# Максимальное число уникальных категорий
max_unique_counts = 150
# Цикл по именам столбцов
for col in melb_df.columns:
    # Проверяем по количеству уникальных значений в этом столбце
    if melb_df[col].nunique() < max_unique_counts and col not in cols_to_exclude:
        # Преобразуем тип столбца
        melb_df[col] = melb_df[col].astype('category')
# смотрим что натворили
melb_df.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

## 2. Сортировка данных в DataFrame  <a class="anchor" id=tt2></a>
[К содержанию](#0)
### МЕТОД SORT_VALUES()
Для сортировки значений в `DataFrame` по значениям одного или нескольких столбцов используется метод `sort_values()`.
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

>Основные параметры метода `sort_values()`
>- `by` — имя или список имён столбцов, по значениям которых производится сортировка.
>- `axis` — ось, по которой производится сортировка (0 — строки, 1 — столбцы). По умолчанию сортировка производится по строкам.
>- `ascending` — сортировка по возрастанию (от меньшего к большему). По умолчанию параметр выставлен на `True`, для сортировки по убыванию (от большего к меньшему) необходимо выставить его на `False`.
>- `ignore_index` — создаются ли новые индексы в таблице. По умолчанию выставлен на `False` и сохраняет индексы изначальной таблицы.
>- `inplace` — производится ли замена исходной таблицы на отсортированную. По умолчанию параметр выставлен на `False`, то есть замены не производится. Чтобы переопределить исходную таблицу на отсортированную, необходимо выставить этот параметр на `True`.

### СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА

Приведём несколько примеров сортировки нашей таблицы с недвижимостью.

Отсортируем таблицу по возрастанию цены объектов недвижимости (`Price`):

In [5]:
# Цена по возрастанию
melb_df.sort_values(by='Price').head(10)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
2652,Footscray,1,unit,85000.0,PI,Burnham,2016-03-09,6.4,3011,1,...,144.89000,Western Metropolitan,7570,42.000000,1.000000,3,9,2,St,0
1805,other,4,house,131000.0,PI,other,2017-02-25,8.9,3162,4,...,145.02420,Southern Metropolitan,2379,17.222222,-0.525994,2,97,5,St,1
7303,Albion,1,unit,145000.0,PI,Biggin,2016-05-28,13.9,3020,2,...,144.82660,Western Metropolitan,2185,31.500000,0.555556,5,46,5,St,1
1927,Coburg,4,house,145000.0,PI,Jellis,2016-04-06,7.8,3058,3,...,144.96580,Northern Metropolitan,11204,20.500000,-0.531429,4,106,2,Rd,0
7940,Hawthorn,1,unit,160000.0,VB,HAR,2017-08-04,4.6,3122,1,...,145.03730,Southern Metropolitan,11308,42.000000,-0.437500,8,8,4,St,0
12666,Brunswick,1,unit,170000.0,VB,Nelson,2017-09-16,5.2,3056,1,...,144.95188,Northern Metropolitan,11918,42.000000,-0.816860,9,47,5,St,1
8811,Footscray,1,unit,170000.0,PI,Burnham,2017-01-07,5.1,3011,1,...,144.89587,Western Metropolitan,7570,8.666667,-0.071429,1,4,5,St,1
8504,West Footscray,1,unit,185000.0,PI,Jas,2017-04-29,8.2,3012,1,...,144.86720,Western Metropolitan,5058,42.000000,1.000000,4,47,5,St,1
7293,Albion,1,unit,185000.0,S,hockingstuart,2016-08-22,13.9,3020,1,...,144.82720,Western Metropolitan,2185,14.333333,-0.964017,8,41,0,Rd,0
7305,Albion,2,unit,190000.0,SP,Burnham,2016-07-30,13.9,3020,2,...,144.82390,Western Metropolitan,2185,25.200000,1.000000,7,46,5,St,1


In [6]:
# Дата по убыванию
melb_df.sort_values(by='Date', ascending=False)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
11144,Northcote,4,house,1955000.0,SP,McGrath,2017-12-08,5.3,3070,4,...,144.99375,Northern Metropolitan,11364,12.600000,0.923664,12,47,4,St,0
11217,Surrey Hills,3,house,1775000.0,PI,Jellis,2017-12-08,10.2,3127,3,...,145.08968,Southern Metropolitan,5457,15.750000,-0.721854,12,47,4,Rd,0
11206,St Kilda,4,house,1600000.0,VB,Gary,2017-12-08,5.0,3182,4,...,144.98324,Southern Metropolitan,13240,16.000000,-0.586028,12,107,4,St,0
11207,Strathmore,7,house,2000000.0,SP,Nelson,2017-12-08,8.2,3041,7,...,144.90587,Western Metropolitan,3284,20.882353,-0.262721,12,18,4,Rd,0
11208,Strathmore,4,house,1610000.0,S,other,2017-12-08,8.2,3041,4,...,144.92243,Western Metropolitan,3284,24.909091,-0.517181,12,62,4,St,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,Camberwell,4,house,2650000.0,S,Jellis,2016-03-09,7.8,3124,4,...,145.06860,Southern Metropolitan,8920,18.900000,-0.550535,3,126,2,Rd,0
4510,Oak Park,3,house,892000.0,S,Nelson,2016-03-09,11.5,3046,3,...,144.91740,Northern Metropolitan,2651,18.000000,-0.714609,3,46,2,St,0
4388,Northcote,3,house,1200000.0,S,Nelson,2016-03-09,5.5,3070,3,...,144.99090,Northern Metropolitan,11364,17.857143,-0.264706,3,96,2,St,0
6184,Surrey Hills,3,house,1205000.0,S,Fletchers,2016-01-28,11.2,3127,3,...,145.10060,Southern Metropolitan,5457,18.000000,-0.590909,1,46,3,Rd,0


### СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ
Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр `by` в виде списка. При этом важно обращать внимание на **порядок следования столбцов.**

Так, например, отсортируем таблицу сначала по возрастанию расстояния от центра города (`Distance`), а затем — по возрастанию цены объекта (`Price`). Для того чтобы вывод был более наглядным, выделим каждую десятую строку из столбцов `Distance` и `Price` результирующей таблицы:

In [7]:
melb_df.sort_values(by=['Distance', 'Price']).loc[::10,['Distance', 'Price']]
# <::10> выводит каждую 10ю строку (шаг среза - 10) 

,Distance,Price
11428,0.0,387000.0
10512,0.7,600000.0
5727,1.2,485000.0
8671,1.2,595000.0
5736,1.2,740000.0
...,...,...
12011,38.0,680000.0
10673,38.0,810000.0
13429,38.0,1155000.0
11102,41.0,650000.0


In [8]:
melb_df.sort_values(by=['Price', 'Distance'], ascending=False).loc[::10,['Price', 'Distance']]
# Cначала сортирует по убыванию цены, а встретив одинаковую цену отсортирует по расстоянию

,Price,Distance
12094,9000000.0,18.8
9179,5510000.0,5.3
7684,5020000.0,7.8
3788,4700000.0,7.4
1050,4250000.0,11.2
...,...,...
7878,260000.0,10.4
7168,250000.0,11.8
2027,241000.0,7.8
5205,216000.0,11.2


### КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ

А теперь рассмотрим применение сортировки на практике.

>Предположим, компания `McGrath` поручила нам восстановить хронологию продаж таунхаусов, у которых площадь участка существенно больше площади здания, чтобы понять, как часто компания справляется с таким сложным видом объектов. Объекты, проданные в один и тот же день, мы бы хотели сортировать по значению коэффициента соотношения площадей. 

Найдём информацию о таунхаусах (`Type`), проданных компанией `(SellerG) McGrath`, у которых коэффициент соотношения площадей здания и участка (`AreaRatio`) меньше `-0.8`. Результат отсортируем по дате продажи (`Date`) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр `ignore_index` на `True`. Для наглядности результата выберем из таблицы только столбцы Data и `AreaRatio`:

In [9]:
mask1 = melb_df['AreaRatio'] < 0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(by=['Date','AreaRatio'], 
                                           ascending=[True, False], 
                                           ignore_index=True).loc[:,['Date','AreaRatio']]

,Date,AreaRatio
0,2016-07-11,0.176849
1,2016-07-26,-0.974922
2,2016-09-24,-0.971831
3,2016-11-27,-0.953608
4,2016-12-11,-0.945946
5,2017-01-07,-0.168317
6,2017-03-06,-0.237013
7,2017-03-06,-0.375000
8,2017-04-03,-0.160000
9,2017-04-03,-0.194631


In [10]:
# 2.2 Произведите сортировку столбца AreaRatio по убыванию. 
# При этом индексы полученной таблицы замените на новые. 
# Какое значение площади здания находится в строке 1558? 
# Ответ округлите до целого числа. 
melb_df.sort_values(by='AreaRatio', ascending=False, 
                    ignore_index=True).iloc[1558].loc['BuildingArea']

126.0

In [11]:
# 2.3 Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. 
# Отсортируйте полученную таблицу сначала по возрастанию числа комнат, 
# а затем по убыванию средней площади комнат (MeanRoomsSquare). 
# Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? 
# Ответ запишите в виде целого числа.
mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
melb_df[mask1 & mask2].sort_values(by=['Rooms', 'MeanRoomsSquare'], 
                    ascending=[True, False], ignore_index=True).iloc[18].loc['Price']

1300000.0

## 3. Группировка данных в DataFrame <a class="anchor" id=tt3></a>
[К содержанию](#0)

Одна из основных задач анализа данных — это группировка данных и **сравнение показателей в группах**. Например, нам необходимо сравнить средний уровень заработной платы в зависимости от пола/уровня образования. Или же мы хотим проследить, какая группа клиентов приносит нам наибольший доход, чтобы направить своё внимание на эту группу.

>В некоторых случаях группировки может быть достаточно, чтобы ответить на вопросы бизнеса. В других случаях это может стать первым шагом в более сложном анализе. Так, например, благодаря группировке мы можем выявлять признаки, которые не несут статистической значимости, или признаки, которые вносят наибольший вклад. 

### МЕТОД GROUPBY() <a class="anchor" id=tt3-1></a>
В библиотеке `Pandas` для группировки данных по одному или нескольким признакам можно использовать метод `groupby()`.
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

>### Основные параметры метода `groupby`
>- `by` — имя или список имён столбцов, по которым производится группировка.
>- `axis` — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
>- `as_index` — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на `True`.

Метод `groupby()` возвращает объект `DataFrameGroupBy`, который хранит в себе информацию о том, какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса. 

>Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (`mean`, `median`, `sum` и т. д.), чтобы рассчитывать показатели внутри каждой группы.

<img src=p_12_img1.png>

Сначала мы разделяем данные на группы с помощью метода `groupby()`, после чего к каждой группе применяем агрегацию и объединяем результаты в новую таблицу.
### ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ <a class="anchor" id=tt3-2></a>
[К содержанию](#0)

Применим агрегирующую функцию среднего к результату работы `groupby()`. В качестве столбца для группировки возьмём столбец типа объекта недвижимости (`Type`):

In [13]:
print(melb_df['Type'].value_counts())
melb_df.groupby(by='Type').mean()

house        9449
unit         3017
townhouse    1114
Name: Type, dtype: int64


,Rooms,Price,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,Lattitude,Longtitude,Propertycount,MeanRoomsSquare,AreaRatio,AgeBuilding
Type,,,,,,,,,,,,,,,
house,3.260874,1.242665e+06,10.979479,3104.080643,3.229336,1.613822,1.772674,617.181924,152.162553,-37.803795,144.994700,7259.025505,18.996731,-0.490031,55.669700
townhouse,2.837522,9.337351e+05,9.851346,3100.777379,2.814183,1.809695,1.555655,279.606822,134.649710,-37.815782,144.996489,7094.459605,18.569847,-0.094916,26.690305
unit,1.963871,6.051275e+05,7.607391,3110.797481,1.966523,1.183295,1.128936,477.314219,102.235863,-37.823710,144.996363,8199.280080,21.068242,0.319883,39.703016


Мы получили таблицу, на пересечении строк и столбцов которой находятся **средние значения** каждого числового признака в наших данных.
>Обратите внимание на структуру получившейся таблицы: теперь на месте индексов стоят значения типа объекта недвижимости `Type` (`house`, `townhouse`, `unit`).

>Примечание. Если мы хотим видеть тип объекта в качестве отдельного столбца таблицы, мы можем выставить параметр `as_index` на `False`:

In [14]:
melb_df.groupby(by='Type', as_index=False).mean()

,Type,Rooms,Price,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,Lattitude,Longtitude,Propertycount,MeanRoomsSquare,AreaRatio,AgeBuilding
0,house,3.260874,1.242665e+06,10.979479,3104.080643,3.229336,1.613822,1.772674,617.181924,152.162553,-37.803795,144.994700,7259.025505,18.996731,-0.490031,55.669700
1,townhouse,2.837522,9.337351e+05,9.851346,3100.777379,2.814183,1.809695,1.555655,279.606822,134.649710,-37.815782,144.996489,7094.459605,18.569847,-0.094916,26.690305
2,unit,1.963871,6.051275e+05,7.607391,3110.797481,1.966523,1.183295,1.128936,477.314219,102.235863,-37.823710,144.996363,8199.280080,21.068242,0.319883,39.703016


Как правило, нам не нужна информация обо всех столбцах, поэтому агрегирующие методы можно применять только к интересующему нас столбцу. Например, давайте сравним средние цены на объекты в зависимости от их типа:

In [15]:
melb_df.groupby(by='Type', as_index=False)['Price'].mean()

,Type,Price
0,house,1.242665e+06
1,townhouse,9.337351e+05
2,unit,6.051275e+05


Примечание. Обратите внимание, что, так как мы считаем только один показатель (среднее) для одного столбца, в результате мы получаем объект `Series`.

Теперь давайте выясним, какие регионы (`Regionname`) наиболее удалены от центра Мельбурна.

Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона. Результат отсортируем по убыванию расстояния:

In [16]:
melb_df.groupby(by='Regionname')['Distance'].min().sort_values(ascending=False)

Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

### ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ <a class="anchor" id=tt3-3></a>
[К содержанию](#0)

Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом `agg()`, который принимает список строк с названиями агрегаций. 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html

Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (`Price`), сгруппированных по номеру месяца продажи (`MonthSale`). Результат отсортируем по количеству продаж в порядке убывания:

In [17]:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

,count,mean,max
MonthSale,,,
8,1850,1.056371e+06,6500000.0
7,1835,9.314698e+05,9000000.0
5,1644,1.097807e+06,8000000.0
6,1469,1.068981e+06,7650000.0
3,1408,1.146762e+06,5600000.0
4,1246,1.050479e+06,5500000.0
9,1188,1.126349e+06,6400000.0
10,854,1.135970e+06,6250000.0
11,750,1.142503e+06,5050000.0


>Примечание. Обратите внимание, что, так как мы считаем несколько показателей для одного столбца, в результате мы получаем объект `DataFrame`.

В результате применения метода `agg()`, в который мы передали список с названиями интересующих нас агрегирующих функций, мы получаем `DataFrame` со столбцами `count`, `mean` и `max`, где для каждого месяца рассчитаны соответствующие параметры. Результат сортируем по столбцу count.

Какие интересные выводы можно сделать из этой таблицы?

- Пик продаж приходится на период весна-лето.

- Средняя цена продаваемых объектов относительно стабильна и находится в пределах 1 млн. австралийских долларов с небольшими отклонениями (около 100 тыс. влево и вправо).

- Прослеживается некоторая зависимость между сезоном и максимальной ценой объектов: в месяцы с большим спросом на объекты недвижимости цена также имеет наибольшие показатели. Можно сделать предположение, что это связано с повышением цен на элитные дома в периоды большого спроса.

>Примечание. Если вам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы, вы можете воспользоваться методом `agg()`, передав в качестве его параметра строку `describe`:

In [18]:
melb_df.groupby('MonthSale')['Price'].agg('describe')

,count,mean,std,min,25%,50%,75%,max
MonthSale,,,,,,,,
1,278.0,9.397921e+05,577668.924214,170000.0,570500.0,795000.0,1111250.0,5200000.0
2,333.0,1.169051e+06,671564.357417,131000.0,710000.0,1020000.0,1478000.0,4735000.0
3,1408.0,1.146762e+06,709573.596867,85000.0,680000.0,945000.0,1400000.0,5600000.0
4,1246.0,1.050479e+06,591892.902979,145000.0,655000.0,905500.0,1298750.0,5500000.0
5,1644.0,1.097807e+06,668492.867996,145000.0,650000.0,905000.0,1371250.0,8000000.0
6,1469.0,1.068981e+06,606010.069052,222000.0,660000.0,900000.0,1325000.0,7650000.0
7,1835.0,9.314698e+05,537390.803161,190000.0,586750.0,800000.0,1150000.0,9000000.0
8,1850.0,1.056371e+06,619617.476541,160000.0,635000.0,892000.0,1310000.0,6500000.0
9,1188.0,1.126349e+06,608734.690742,170000.0,725000.0,980000.0,1360000.0,6400000.0


После базовых математических функций наиболее частым агрегированием является подсчёт числа **уникальных значений**. Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода `agg()` строку `nunique`. 

Более того, метод `agg()` поддерживает использование и других функций. Передадим дополнительно встроенную функцию `set`, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:

In [19]:
melb_df.groupby(by='Regionname')['SellerG'].agg(['nunique', set])

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{RW, Kay, Stockdale, other, Purplebricks, McGr..."
Eastern Victoria,11,"{Ray, HAR, Eview, Harcourts, hockingstuart, C2..."
Northern Metropolitan,40,"{RW, Kay, Village, Collins, LITTLE, Brad, Greg..."
Northern Victoria,11,"{Ray, HAR, McDonald, hockingstuart, Barry, LIT..."
South-Eastern Metropolitan,25,"{RW, Chisholm, Greg, Stockdale, O'Brien, Purpl..."
Southern Metropolitan,38,"{RW, Kay, Chisholm, Collins, LITTLE, Greg, Sto..."
Western Metropolitan,34,"{RW, Village, Chisholm, Brad, Greg, Stockdale,..."
Western Victoria,6,"{Ray, HAR, hockingstuart, Raine, YPA, other}"


Как и ожидалось, наименьшая конкуренция в наиболее удалённом регионе Western Victoria, а наибольшая — в центральном районе Northern Metropolitan.

## Практика по группировкам данных <a class="anchor" id=tt3-4></a>
[К содержанию](#0)

In [20]:
# 3.1 Сгруппируйте данные по признаку количества комнат и 
# найдите среднюю цену объектов недвижимости в каждой группе. 
# запишите количество комнат, для которых средняя цена наибольшая.
melb_df.groupby('Rooms')['Price'].agg(
    ['mean']).sort_values(by='mean', ascending=False)

,mean
Rooms,
7,1.920700e+06
5,1.870260e+06
6,1.849366e+06
8,1.602750e+06
4,1.445282e+06
3,1.076081e+06
10,9.000000e+05
2,7.750812e+05
1,4.338245e+05


In [21]:
# 3.2 Какой регион имеет наименьшую протяжённость по географической широте (Lattitude)?
# Для ответа на этот вопрос рассчитайте стандартное отклонение широты для каждого региона.
# В качестве ответа запишите название этого региона.
melb_df.groupby(by='Regionname')['Lattitude'].agg(['std']).sort_values(by='std')

,std
Regionname,
Western Victoria,0.011579
Southern Metropolitan,0.043080
Eastern Metropolitan,0.047890
Northern Metropolitan,0.049639
Western Metropolitan,0.051251
South-Eastern Metropolitan,0.073411
Northern Victoria,0.084455
Eastern Victoria,0.147067


In [22]:
# 3.3 Какая риелторская компания (SellerG) имеет наименьшую общую выручку 
# за период с 1 мая по 1 сентября (включительно) 2017 года? 
# Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период. 
# Не забудьте перевести даты в формат datetime.
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
mask1 = melb_df['Date'].dt.year == 2017
mask2 = melb_df['Date'].dt.month >= 5
mask3 = melb_df['Date'].dt.month >= 8
#melb_df[mask1 & mask2 & mask3].groupby(by='SellerG')['Price'].agg('describe')
melb_df[mask1 & mask2 & mask3].groupby(by='SellerG')['Price'].agg(
    ['sum']).sort_values(by='sum')

,sum
SellerG,
LITTLE,800000.0
Thomson,1080000.0
Alexkarbon,1270000.0
Moonee,2365000.0
Burnham,2375000.0
Rendina,2640000.0
McDonald,3168000.0
Bells,4920000.0
Chisholm,6869000.0


## 4. Сводные таблицы <a class="anchor" id=tt4></a>
[К содержанию](#0)

Сводная таблица принимает на вход данные из отдельных столбцов и группирует их. В результате получается **новая таблица**, которая позволяет увидеть многомерное обобщение данных. Таким образом, благодаря сводным таблицам мы можем оценить зависимость между двумя и более признаками данных.

>Мы чаще сталкиваемся со сводными таблицами, чем с обычными, в плоском виде (подразумевается, что индексами являются номера строк, а столбцами — имена столбцов.), так как сводные таблицы удобнее для анализа и быстрых выводов, а также позволяют увидеть более общие зависимости между признаками, нежели простая группировка данных.

### МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ <a class="anchor" id=tt4-1></a>
На самом деле мы с вами уже строили простейшие одномерные сводные таблицы с помощью метода groupby — мы рассматривали сводную таблицу в контексте группировки по одному признаку. 

Например, мы уже умеем строить таблицу, которая показывает зависимость медианной цены и площади здания от числа комнат:


In [23]:
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].quantile(0.5)

,Price,BuildingArea
Rooms,,
1,385000.0,107.0
2,690000.0,126.0
3,950000.0,126.0
4,1285000.0,142.0
5,1660000.0,176.0
6,1800000.0,126.0
7,1496000.0,216.5
8,1515000.0,126.0
10,900000.0,126.0


Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (`Type`). Для этого в параметрах метода `groupby()` укажем список из нескольких интересующих нас столбцов.

In [24]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean()

Rooms  Type     
1      house        8.668655e+05
       townhouse    5.927045e+05
       unit         3.899289e+05
2      house        1.017238e+06
       townhouse    7.101585e+05
       unit         6.104905e+05
3      house        1.109233e+06
       townhouse    9.847087e+05
       unit         8.505963e+05
4      house        1.462283e+06
       townhouse    1.217092e+06
       unit         1.037476e+06
5      house        1.877327e+06
       townhouse    1.035000e+06
       unit                  NaN
6      house        1.869508e+06
       townhouse             NaN
       unit         5.200000e+05
7      house        1.920700e+06
       townhouse             NaN
       unit                  NaN
8      house        1.510286e+06
       townhouse             NaN
       unit         2.250000e+06
10     house        9.000000e+05
       townhouse             NaN
       unit                  NaN
Name: Price, dtype: float64

В результате выполнения такого кода мы получаем `Series`, которая обладает несколькими уровнями индексов: первый уровень — число комнат, второй уровень — тип здания. Такая организация индексов называется **иерархической**. Вычисление параметра (средней цены) происходит во всех возможных комбинациях признаков.
<a class="anchor" id=4-1></a>

Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам, а второй — по столбцам), а не в виде `Series` с иерархическими индексами, к результату чаще всего применяют метод `unstack()`, который позволяет переопределить вложенный индекс в виде столбцов таблицы:

In [25]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack().round()

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,NaN
6,1869508.0,NaN,520000.0
7,1920700.0,NaN,NaN
8,1510286.0,NaN,2250000.0
10,900000.0,NaN,NaN


В результате мы получаем сводную таблицу, столбцы в которой представляют типы домов (`house`, `townhouse`, `unit`), строки — число комнат, а на пересечении строк и столбцов находится средняя стоимость объекта с такими показателями.
- Пропуски в сводной таблице (NaN) говорят о том, что в наших данных нет соответствующих комбинаций признаков. Например, у нас нет информации о ценах на таунхаусы, где количество комнат больше пяти.
- Наибольшей средней стоимостью (2,25 млн. австралийских долларов) обладают объекты типа unit с восемью жилыми комнатами. Наименьшая средняя стоимость — у однокомнатных домов типа unit (чуть меньше 400 тыс. австралийских долларов).
- Сколько бы комнат ни было в доме, цена на объекты типа unit всегда ниже других (за исключением восьмикомнатных объектов).


### МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ <a class="anchor" id=tt4-2></a>
[К содержанию](#0)

На самом деле метод groupby редко используется при двух параметрах, так как для построения сводных таблиц существует специальный и более простой метод — `pivot_table()`.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html

>Основные параметры метода `pivot_table()`
>- `values` — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
>- `index` — имя столбца, значения которого станут строками сводной таблицы;
>- `columns` — имя столбца, значения которого станут столбцами сводной таблицы;
>- `aggfunc` — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
>- `fill_value` — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

Давайте построим ту же самую таблицу, но уже с использованием метода `pivot_table`. В качестве параметра `values` укажем столбец `Price`, в качестве индексов сводной таблицы возьмём `Rooms`, а в качестве столбцов — `Type`. Агрегирующую функцию оставим по умолчанию (среднее). Дополнительно заменим пропуски в таблице на значение 0. Финальный результат для наглядности вывода округлим с помощью метода `round()` до целых.

In [26]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


>Несложно понять, что метод `pivot_table()` имеет преимущество перед группировкой по нескольким критериям. Оно заключается в наличии специальных аргументов для строк и столбцов сводной таблицы, благодаря чему уменьшается вероятность запутаться при построении более сложных (многомерных) сводных таблиц, о которых мы поговорим далее.

А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. Для этого построим сводную таблицу, в которой строками будут являться названия регионов (`Regionname`), а в столбцах будет располагаться наш «признак-мигалка» выходного дня (`Weekend`), который равен 1, если день был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

In [27]:
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


- Число продаж резко возрастает в выходные вне зависимости от региона (приблизительно в 2-3 раза). То есть вероятность того, что дом продадут в выходные, гораздо выше вероятности, что его продадут в будний день.

- В отдалённых регионах (Victoria) коэффициент роста числа продаж выше, чем в центральных. Если в центральных регионах Metropolitan продажи по выходным в 2-2.5 раза выше, чем по будням, то в регионах Victoria число продаж в выходные вырастает примерно в 3 раза.

- Такой рост можно даже попытаться объяснить логически: в выходные дни у людей появляется свободное время, чтобы доехать до отдалённых пригородов с целью покупки дома.

Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (`Landsize`) от типа объекта (`Type`) и его региона (`Regionname`). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент `aggfunc` список из агрегирующих функций. Построим такую сводную таблицу, где пропущенные значения заменим на 0:

In [28]:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

median                        mean              \
Type                        house townhouse unit        house   townhouse   
Regionname                                                                  
Eastern Metropolitan        674.0     233.5  203   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62   507.883406  244.560669   
Western Victoria            599.5       0.0    0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

>Обратите внимание на добавление дополнительных индексов столбцов `median` и `mean`. Здесь медианное и среднее значения рассчитаны отдельно для каждой комбинации признаков.

Здесь в глаза бросаются объекты типа house в регионах Eastern Victoria и Northern Victoria — в них среднее и медиана отличаются более чем в три раза. Вероятно, это связано с тем, что в этих районах очень большой разброс цен: есть несколько объектов с гигантской площадью, а остальные объекты имеют небольшую площадь. Из-за этого среднее значение искажается, в то время как медиана нечувствительна к такому разбросу и не искажает результат.

## Многомерные сводные таблицы <a class="anchor" id=tt4-3></a>
[К содержанию](#0)

До этого мы рассматривали, как некоторый статистический показатель может зависеть от двух признаков. Однако, как уже упоминалось, сводные таблицы позволяют наблюдать зависимость и от большего числа признаков. Такие сводные таблицы называются **многомерными**.

>Для того чтобы исследовать зависимость от большего числа признаков, можно передать список признаков в параметр `index` или параметр `columns`.

Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (`Method`) и типа объекта (`Type`), по столбцам — наименование региона (`Regionname`), а на пересечении строк и столбцов будет стоять медианная цена объекта (`Price`):

In [29]:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                   1244000            780000   
       townhouse                760000                 0   
       unit                     650000                 0   
S      house                   1127000            675000   
       townhouse                828000                 0   
       unit                     645750            492000   
SA     house                    932500            950000   
       townhouse                807500                 0   
       unit                          0                 0   
SP     house                   1050000            672500   
       townhouse                910000                 0   
       unit                     515000            400000   
VB     house                   1100000            712500   
       townhouse                892500                 0   
       unit                     500000                 0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                     900000             500000   
       townhouse                 632500                  0   
       unit                      410000                  0   
S      house                     920000             555000   
       townhouse                 750000                  0   
       unit                      525500                  0   
SA     house                     817500             540000   
       townhouse                 425000                  0   
       unit                      616000                  0   
SP     house                     900000             521000   
       townhouse                 690000                  0   
       unit                      470000                  0   
VB     house                    1050000             690000   
       townhouse                 640000                  0   
       unit                      450000                  0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                          865000                1725000   
       townhouse                     1190000                1055000   
       unit                           525000                 571250   
S      house                          883300                1611000   
       townhouse                      875000                1135000   
       unit                           606000                 655000   
SA     house                          880000                1390000   
       townhouse                           0                1141000   
       unit                                0                 580000   
SP     house                          770000                1521750   
       townhouse                      800000                1162500   
       unit                           601000                 550000   
VB     house                          850000                1800000   
       townhouse                           0                1250000   
       unit                           700000                 500000   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                    870000            630000  
       townhouse                670000                 0  
       unit                     360000                 0  
S      house                    870000            397500  
       townhouse                729000                 0  
       unit                     489000                 0  
SA     house                    772500                 0  
       townhouse                467500                 0  
       unit                     571000                 0  
SP     house                    865000            360000  
       t

Первым индексом в таблице идёт метод продажи здания, далее для метода указывается тип недвижимости. По столбцам расположены регионы. В ячейках таблицы указана медианная цена для каждой такой комбинации.

Такие таблицы уже сложнее читать, однако с помощью них можно более глубоко исследовать закономерности. Например, можно видеть, что вне зависимости от метода продажи и региона цена на объекты типа house практически всегда выше, чем на объекты другого типа.

## Доступ к данным в сводной таблице <a class="anchor" id=tt4-4></a>
[К содержанию](#0)

Как получить доступ к данным или произвести фильтрацию в сложной сводной таблице, где есть дополнительные индексы?

Давайте рассмотрим, что собой представляют столбцы сложной сводной таблицы.

Запишем сводную таблицу, которую мы создавали ранее в переменную `pivot`:

In [30]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot


median                        mean              \
Type                        house townhouse unit        house   townhouse   
Regionname                                                                  
Eastern Metropolitan        674.0     233.5  203   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62   507.883406  244.560669   
Western Victoria            599.5       0.0    0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

In [31]:
# Выведем столбцы
pivot.columns

MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

>В результате мы получаем объект `MultiIndex`. Этот объект хранит в себе шесть комбинаций пар столбцов (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов обращения к столбцам таблицы.

>Мультииндексы раскрываются подобно вложенным словарям — по очереди, как матрёшка. Чтобы получить доступ к определённому столбцу, вы должны сначала обратиться к столбцу, который находится уровнем выше.

Так, из таблицы `pivot` мы можем получить средние значения площадей участков для типа здания `unit`, просто последовательно обратившись по имени столбцов:

In [32]:
pivot['mean']['unit']

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

Аналогично производится и фильтрация данных. Например, если нам нужны регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:

In [33]:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
filtered_pivot

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом `index` и обернуть результат в список:

In [34]:
print(list(filtered_pivot.index))

['Southern Metropolitan', 'Western Metropolitan']


Таким образом, сводные таблицы изначально кажутся сложной структурой, но на самом деле это обычные `DataFrame` со вложенными индексами строк или столбцов. 

Умение читать и анализировать сложные сводные таблицы — это важный навык, который помогает проводить углублённый анализ данных.

>Примечание. На самом деле мультииндексные таблицы можно создавать и вручную. Давайте посмотрим на синтаксис данной конструкции:

In [35]:
import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
mser

white  up       0.801823
       down     0.117220
       right    0.507551
blue   up       0.880092
       down     0.939995
red    up       0.929226
       down     0.162678
       left     0.732252
dtype: float64

В данном примере мы создаём объект `Series` со вложенными индексами. Мы передаём в качестве индексов `Series` вложенный список, где первый список задаёт внешний уровень вложенности, а второй список — внутренний уровень вложенности. Значения `Series` — случайные числа от 0 до 1, сгенерированные функцией `np.random.rand() `(ваши значения могут отличаться).

Если посмотреть на индексы `Series`, можно увидеть, что они являются мультииндексами:

In [36]:
mser.index

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )

Аналогично создаются `DataFrame` со вложенными признаками (вложенными столбцами) — для этого вложенный список передаётся в параметр columns при инициализации таблицы:

In [37]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
mframe

pen               paper          
                   1         2         1         2
white up   -0.011873  0.059886 -0.596484  0.327451
      down  0.293737  0.435791  0.034073 -0.059071
red   up   -0.697687 -0.513759  0.174626  0.406842
      down  1.536241  0.153944  0.750777 -1.548714

## Практика по сводным таблицам <a class="anchor" id=tt4-5></a>
[К содержанию](#0)

Какой параметр метода `pivot_table()` отвечает за признак, по которому будут рассчитаны агрегирующие функции?

`aggfunc`

In [38]:
# 4.2 Составьте сводную таблицу, которая показывает зависимость медианной площади 
# (BuildingArea) здания от типа объекта недвижимости (Type) 
# и количества жилых комнат в доме (Rooms). 
# Для какой комбинации признаков площадь здания наибольшая? 
# запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.
melb_df.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    aggfunc='median',
    fill_value=0
)

Type,house,townhouse,unit
Rooms,,,
1,126.0,88.0,69.5
2,126.0,114.0,110.0
3,126.0,126.0,126.0
4,141.0,159.5,126.0
5,177.0,152.0,0.0
6,126.0,0.0,171.0
7,216.5,0.0,0.0
8,126.0,0.0,126.0
10,126.0,0.0,0.0


In [39]:
# 4.3 Составьте сводную таблицу, которая показывает зависимость 
# средней цены объекта недвижимости (Price) от риелторского агентства (SellerG) 
# и типа здания (Type). Во вновь созданной таблице найдите агентство, 
# у которого средняя цена для зданий типа unit максимальна. 
# В качестве ответа запишите название этого агентства
melb_df.pivot_table(
    values='Price',
    columns='Type',
    index='SellerG',
    aggfunc='mean'
).sort_values(by=['unit'], ascending=False)

Type,house,townhouse,unit
SellerG,,,
Nick,2.111984e+06,7.800000e+05,993642.857143
Kay,2.471453e+06,1.648818e+06,851660.606061
Cayzer,1.608258e+06,1.277333e+06,832333.333333
Marshall,2.161028e+06,1.590250e+06,770044.678261
Philip,1.068569e+06,7.370000e+05,706255.555556
RT,1.841567e+06,1.261643e+06,703509.803922
C21,9.693400e+05,7.687500e+05,700700.000000
Fletchers,1.480143e+06,1.248796e+06,691625.000000
Williams,1.258358e+06,8.587143e+05,685386.486486


## 5. Объединение DataFrame: знакомимся с новыми данными <a class="anchor" id=tt5></a>
[К содержанию](#0)

На практике источники данных редко ограничиваются одной таблицей. Например, если мы работаем с базой данных, то данные в ней могут быть представлены в виде нескольких десятков таблиц, каждая из которых несёт отдельную информацию. Если вы делаете выгрузку из базы напрямую, не объединяя таблицы в единую структуру средствами `SQL`, вам необходимо знать, как работать с такими таблицами средствами Pandas. 

В этой части модуля мы будем работать с популярным датасетом **MovieLens**, в котором собраны логи некоторой рекомендательной системы фильмов.

https://grouplens.org/datasets/movielens/

Наши данные представляют собой четыре таблицы:

- `ratings1` и `ratings2` — таблицы с данными о выставленных пользователями оценках фильмов. Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов.
>- `userId` — уникальный идентификатор пользователя, который выставил оценку;
>- `movieId` — уникальный идентификатор фильма;
>- `rating` — рейтинг фильма

- `dates` — таблица с датами выставления всех оценок.
>- `date` — дата и время выставления оценки фильму.

- `movies` — таблица с информацией о фильмах
>- `movieId` — уникальный идентификатор фильма;
>- `title` — название фильма и год его выхода;
>- `genres` — жанры фильма.

### ЗАЧЕМ ХРАНИТЬ ДАННЫЕ В РАЗНЫХ ТАБЛИЦАХ?
Конечно, здорово, если все необходимые данные лежат в одной таблице, но на практике такое случается редко по двум объективным причинам:
 - Часто данные формируются **несколькими независимыми процессами**, каждый из которых хранит данные в своей таблице
 - Хранить все данные в одной таблице часто очень **накладно для ёмкости диска**.

Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:

1. Склеим (конкатенируем) таблицы ratings1 и ratings2 в единую структуру.

2. К полученной таблице с рейтингами подсоединим столбец с датой проставления рейтинга, склеив столбцы таблиц между собой.

3. Присоединим к нашей таблице информацию о названиях и жанрах фильмов.


In [2]:
ratings1 = pd.read_csv('ratings1.csv', sep=',')
ratings2 = pd.read_csv('ratings2.csv', sep=',')
dates = pd.read_csv('dates.csv')
movies = pd.read_csv('movies.csv')
dates['date'] = pd.to_datetime(dates['date'], dayfirst=False)
dates['year'] = dates['date'].dt.year
dates.groupby(by='year')['year'].count().sort_values(ascending=False)

year
2000    10061
2017     8198
2007     7114
2016     6703
2015     6616
2018     6418
1996     6040
2005     5813
2012     4656
2008     4351
2009     4158
2006     4059
2003     4014
2001     3922
2002     3478
2004     3279
1999     2439
2010     2301
1997     1916
2011     1690
2013     1664
2014     1439
1998      507
Name: year, dtype: int64

## 6. Объединение DataFrame: concat <a class="anchor" id=tt6></a>
[К содержанию](#0)

Следуя нашему плану объединения таблиц, первым делом мы должны склеить таблицы `ratings1` и `ratings2` по строкам.

Для этого воспользуемся встроенной функцией Pandas `concat()`, которая позволяет склеивать (конкатенировать) таблицы как по строкам, так и по столбцам.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

### Основные параметры функции `concat()`
>- `objs` — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
>- `axis` — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
>- `join` — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
>- `ignore_index` — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.

Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.

Итак, давайте склеим  `ratings1` и `ratings2` по строкам, так как они имеют одинаковую структуру столбцов. Для этого передадим их списком в функцию `concat()`. Помним, что параметр `axis` по умолчанию равен 0, объединение происходит по строкам, поэтому не трогаем его.

>Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом `DataFrame`. Поэтому её вызов осуществляется как `pd.concat(...)`.

In [41]:
ratings = pd.concat([ratings1, ratings2])
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0


>На первый взгляд может показаться, что всё прошло успешно, однако если мы посмотрим на индексы последних строк таблицы, то увидим, что их нумерация не совпадает с количеством строк. Это может привести к некорректному объединению таблиц по ключевым столбцам на следующем этапе решения нашей задачи.

Это связано с тем, что по умолчанию `concat` сохраняет первоначальные индексы объединяемых таблиц, а обе наши таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр `ignore_index` на `True`:

In [9]:
ratings = pd.concat([ratings1, ratings2], ignore_index=True)
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах `ratings` и `dates`, ведь нам предстоит вертикально склеить их между собой:

In [10]:
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


Сука! Размерности не совпадают.

На самом деле очень просто: при выгрузке данных информация об оценках какого-то  пользователя попала в обе таблицы (`ratings1` и `ratings2`). В результате конкатенации случилось дублирование строк. В данном примере их легко найти — выведем последнюю строку таблицы `ratings1` и первую строку таблицы `ratings2`:

In [11]:
print(ratings1.tail(1))
print(ratings2.head(1))

       userId  movieId  rating
40000     274     5621     2.0
   userId  movieId  rating
0     274     5621     2.0


**Чтобы очистить таблицу от дублей**, мы можем воспользоваться методом `DataFrame` `drop_duplicates()`, который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр `ignore_index` в методе `drop_duplicates()` на значение `True`:

In [12]:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


Наконец, мы можем добавить к нашей таблице с оценками даты их выставления. Для этого конкатенируем таблицы `ratings` и `dates` **по столбцам**:

In [13]:
ratings_dates = pd.concat([ratings, dates], axis=1)
ratings_dates

,userId,movieId,rating,date,year
0,1,1,4.0,2000-07-30 18:45:03,2000
1,1,3,4.0,2000-07-30 18:20:47,2000
2,1,6,4.0,2000-07-30 18:37:04,2000
3,1,47,5.0,2000-07-30 19:03:35,2000
4,1,50,5.0,2000-07-30 18:48:51,2000
...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,2017
100832,610,168248,5.0,2017-05-03 22:21:31,2017
100833,610,168250,5.0,2017-05-08 19:50:47,2017
100834,610,168252,5.0,2017-05-03 21:19:12,2017


Итак, мы смогли создать единую таблицу с рейтингами и датами их представления. Нашим следующим шагом будет присоединить к таблице информацию о фильмах из таблицы `movies`.

Напишем функцию объединяющую однотипные (одинаковой размерности) таблицы в результирующий `DateFrame`. Объединять будем построчно, с удалением дубликатов и присвоением новых индексов.

In [4]:
def concat_users_files(path):
    """Объединятель одноразмерных таблиц из указанной директории,
    объединение по строкам вниз с присвоением новых индексов

    Args:
        path (_str_): путь до директории с файлами

    Returns:
        _pandas.core.frame.DataFrame_: таблица с объединёнными данными
    """
    # создаём пустой df
    df = pd.DataFrame()
    # получаем список из имён файлов
    names_list = os.listdir(path) 
    names_list.sort()
    # циклом проходимся по именам файлов для конкатенации
    for csv_file in names_list:
    # проводим чтение файлов соединяя путь до директории с именем файла
        df = pd.concat(
            [df,pd.read_csv(path + csv_file)], ignore_index=True)
    # удаляем дубли, обновляем индексы    
    df = df.drop_duplicates(ignore_index=True)
    # возвращаем получившийся df
    return df

data = concat_users_files()
print(type(data))
data
        

TypeError: concat_users_files() missing 1 required positional argument: 'path'

## 7. Объединение DataFrame: join, merge <a class="anchor" id=tt7></a>
[К содержанию](#0)

У таблиц `ratings` и `movies` есть общий столбец `movieId`, который каждому фильму из таблицы `movies` ставит в соответствие поставленные ему оценки из таблицы `ratings`. Мы хотим объединить их в единую структуру согласно этому соответствию. Объединения такого рода часто называют **объединением по ключевому столбцу**.

>Однако прежде чем мы перейдём к дальнейшей работе с нашими таблицами о фильмах, мы должны рассмотреть основные типы объединения таблиц.

<img src=p_12_img4.png>

### ТИПЫ ОБЪЕДИНЕНИЙ

Типы объединений в `Pandas` тесно связаны с операцией `join` из `SQL`, которую мы будем рассматривать в курсе в дальнейшем.

Они представлены на схеме ниже в виде **кругов Эйлера**. 

<img src=p_12_img2.png>

>Круги Эйлера — это геометрический способ отобразить отношения между множествами. Мы уже сталкивались с базовыми операциями между множествами (пересечением, объединением и вычитанием), когда говорили о множествах в модуле PYTHON-2.

Прежде чем мы перейдём к дальнейшей работе с таблицами о фильмах, рассмотрим два основных типа объединения таблиц:

>**inner** (внутреннее)
>
>При использовании такого типа объединения в результирующей таблице остаются только те записи, которые есть в обеих таблицах.
>
>Аналогия в теории множеств
>Пересечение (`intersection`) множеств А и В.
>
>**Строки, для которых совпадение не было найдено, удаляются.**

>**outer** (внешнее)
>
>Данный тип делится на три подтипа:
>
>`full` — используется как `outer` по умолчанию, объединяет все варианты в обеих таблицах.
>
>Аналогия в теории множеств
Объединение (`union`) множеств А и В.
>
>`left` — для всех записей из «левой» таблицы (например, `ratings`) ведётся поиск соответствий в «правой» (например, `movies`). В результирующей таблице останутся только те значения, которым были найдены соответствия, то есть только значения из `ratings`.
>
>Аналогия в теории множеств
Вычитание (`difference`) множества B из результата объединения (`union`) множеств А и В.
>
>`right` — аналогично предыдущему, но остаются значения только из «правой» таблицы. 
>
>Аналогия в теории множеств
Вычитание (`difference`) множества А из результата объединения (union) множеств А и В.
>
>Во всех трёх случаях, если совпадений между таблицами не найдено, на этом месте ставится пропуск (`NaN`).

## Метод объединения join() <a class="anchor" id=tt7-1></a>
[К содержанию](#0)

Для объединения двух таблиц по индексам используется метод `DataFrame` `join()`. Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу (в нашем случае это `movieId`).

<img src=p_12_img3.png>

>Основные параметры метода `join()`
>
>`other` — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».
>
>`how` — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.
>
>`on` — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».
>
>`lsuffix` и `rsuffix` — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

>Если использовать метод `join()` «в лоб» (без указания ключевого столбца), то объединение произойдёт, как и задумано — по индексам двух таблиц согласно установленному типу объединения.

Проверим это, объединив таблицы типом `left`. Так как в наших таблицах есть одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки:

In [14]:
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
joined_false

,userId,movieId,rating,date,year,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,2000,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,2000,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,2000,5.0,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,2017,NaN,NaN,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,2017,NaN,NaN,NaN
100833,610,168250,5.0,2017-05-08 19:50:47,2017,NaN,NaN,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,2017,NaN,NaN,NaN


При объединении таблиц по индексам в результирующую таблицу попали все строки из «левой» таблицы, а недостающие строки из «правой» были заполнены пропусками. Так работает тип объединения `left`.

>Обратите внимание, что в данном случае у нас получилось два столбца, соответствующих идентификатору фильма: один — из «левой» таблицы (`movieId`), а другой — из «правой» (`movieId_right`).

Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. Чтобы совместить таблицы по ключевому столбцу с помощью метода `join()`, необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью метода `set_index()`. Также необходимо указать название ключа в параметре on.

In [ ]:
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
joined.head()

,userId,movieId,rating,date,year,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,2000,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,2000,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,2000,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


В результате такого объединения для каждого идентификатора фильма `movieId` в таблице `ratings_dates` найден совпадающий с ним идентификатор `movieId` в таблице `movies` и присоединена информация о самом фильме (`title` и `genres`). Это как раз то, что нам нужно.
>Обратите внимание, что в результате такого объединения остался лишь один столбец `movieId`.

## Метод объединения merge() <a class="anchor" id=tt7-2></a>
[К содержанию](#0)

Аналогично предыдущему, метод `merge()` предназначен для слияния двух таблиц по ключевым столбцам или по индексам. Однако, в отличие от `join()`, метод `merge()` предлагает более **гибкий способ** управления объединением, благодаря чему является более популярным.

>Основные параметры метода `merge()`
>
>- `right` — присоединяемая таблица. По умолчанию она является «правой».
>- `how` — параметр типа объединения. По умолчанию принимает значение 'inner'.
>- `on` — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную.
>- `left_on` — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы.
>- `right_on` — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.
>- `lsuffix` и `rsuffix` — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

Посмотрим на метод `merge()` в действии. Произведём слияние наших таблиц и получим ту же таблицу, что и ранее:

In [ ]:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
merged.head()

,userId,movieId,rating,date,year,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,2000,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,2000,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,2000,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Проверим, что число строк в таблице `ratings_dates` совпадает с числом строк в результирующей таблице `merged`:

In [ ]:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


## Особенности метода merge()

Возникает вопрос: почему мы выбрали тип объединения `left`, а не `full`, например?

Найти ответ нам поможет пример. Объединим `ratings_dates` с movies по ключевому столбцу `movieId`, но с параметром `how='outer'` (`full outer`) и выведем размер таблицы, а также её «хвост»:

In [ ]:
merged2 = ratings_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице movies: ', movies.shape[0])
print('Число строк в таблице merged2: ', merged2.shape[0])
merged2.tail()

Число строк в таблице ratings_dates:  100836
Число строк в таблице movies:  9742
Число строк в таблице merged2:  100854


,userId,movieId,rating,date,year,title,genres
100849,NaN,30892,NaN,NaT,NaN,In the Realms of the Unreal (2004),Animation|Documentary
100850,NaN,32160,NaN,NaT,NaN,Twentieth Century (1934),Comedy
100851,NaN,32371,NaN,NaT,NaN,Call Northside 777 (1948),Crime|Drama|Film-Noir
100852,NaN,34482,NaN,NaT,NaN,"Browning Version, The (1951)",Drama
100853,NaN,85565,NaN,NaT,NaN,Chalet Girl (2011),Comedy|Romance


Результирующее число строк в таблице увеличилось. Но за счёт чего?

Оказывается, в таблице `movies` содержались фильмы, которым ещё не были выставлены оценки. В результате объединения типом `full outer` информация о фильмах перенеслась из таблицы `movies` в результирующую таблицу. Однако, поскольку оценки фильмам ещё не были выставлены, соответствующие столбцы таблицы `ratings_dates` заполнились пропусками (`NaN`). Такие фильмы были записаны в конец таблицы.

>Важно! Учитывайте такие нюансы при работе с несколькими таблицами и всегда проверяйте результат объединения.

Метод `merge()` с внешним (`outer`) типом объединения может использоваться как аналог метода `concat()` при объединении таблиц с одинаковой структурой (одинаковые количество и названия столбцов) по строкам. В таком случае все одноимённые столбцы таблиц будут считаться ключевыми.

Рассмотрим пример: объединим таблицы `ratings1` и `ratings2`, как мы уже делали раньше, но теперь используем метод `merge()`:

In [ ]:
merged3 = ratings1.merge(
    ratings2,
    how='outer')
print(merged3)
print(ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0

[100836 rows x 3 columns]
        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0

[100836 rows x 3 columns]


>Обратите внимание, что при использовании метода `merge()` для склейки двух таблиц у нас автоматически **пропали дубликаты**, которые мы видели при использовании метода `concat()`. Это особенность метода `merge()` — автоматическое удаление дублей.

In [ ]:
# 7.3
data_1 = pd.DataFrame({'Value': [100, 45, 80],
                       'Group': [1, 4, 5]},
                      index = ['I0', 'I1', 'I2']
                     )

data_2 = pd.DataFrame({'Company': ['Google', 'Amazon', 'Facebook'],
                       'Add': ['S0', 'S1', 'S7']},
                      index = ['I0', 'I1', 'I3']
                     )
print(data_1)
print(data_2)
joined = data_1.join(
    data_2,
    #how = 'left'
    #how = 'right'
    how = 'inner'
    #how = 'outer' 
)
# how = 'left' - по индесам из левой таблицы (недостающие значения - NaN)
# how = 'right' - по индесам из правой таблицы (недостающие значения - NaN)
# how = 'inner' - по пересекающимся индексам
# how = 'outer' - склеит вообще всё (недостающие значения - NaN)
joined

    Value  Group
I0    100      1
I1     45      4
I2     80      5
     Company Add
I0    Google  S0
I1    Amazon  S1
I3  Facebook  S7


,Value,Group,Company,Add
I0,100,1,Google,S0
I1,45,4,Amazon,S1


In [ ]:
# 7.4
a = pd.DataFrame({'A': ['a', 'b', 'c'], 'B': [103, 214, 124], 'C': [1, 4, 2]})
b = pd.DataFrame({'V': ['d', 'b', 'c'], 'U': [1393.7, 9382.2, 1904.5], 'C': [1, 3, 2]})
print(a)
print(b)
joined = a.merge(
    b,
    on='C',
    #how = 'left'
    how = 'right'
    #how = 'inner'
    #how = 'outer' 
)
joined

   A    B  C
0  a  103  1
1  b  214  4
2  c  124  2
   V       U  C
0  d  1393.7  1
1  b  9382.2  3
2  c  1904.5  2


,A,B,C,V,U
0,a,103.0,1,d,1393.7
1,NaN,NaN,3,b,9382.2
2,c,124.0,2,c,1904.5


In [ ]:
#7.5 
# информация по складу
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})
# информация по продажам
purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})
# объединение по itemId только тех позиций, по которым были продажи
merged = items_df.merge(
    purchase_df,
    on='item_id',
    how='inner')
# добавим столбец с стоимостью
merged['total'] = merged['stock_count'] * merged['price']
# посчитаем возможную выручку
income = merged['total'].sum()

print(merged)
print('Суммарная возможная выручка:',income)

   item_id   vendor  stock_count  purchase_id  price    total
0   417283  Samsung           54          101  13900   750600
1   849734       LG           33          101   5330   175890
2   132223    Apple          122          101  38200  4660400
3   573943    Apple           18          112  49990   899820
4    19475       LG          102          121   9890  1008780
5  3294095    Apple           43          145  33000  1419000
6   382043  Samsung           77          145  67500  5197500
7   302948  Samsung          143          145  34500  4933500
8   100132       LG           60          221  11400   684000
Суммарная возможная выручка: 19729490


## 7. Практика. Закрепление знаний <a class="anchor" id=tt8></a>
[К содержанию](#0)

Для решения задач нам понадобится выделить из признака `title` год выпуска фильма. Для этого напишем функцию `get_year_release(arg)`.

In [15]:
ratings_movies = pd.read_csv('ratings_movies.csv', sep=',')
ratings_movies

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


In [16]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

Данная функция основана на регулярных выражениях и использует модуль `re`, с которым мы пока не работали.

>Модуль `re` предназначен для поиска шаблонов в тексте и встроен в язык, поэтому не нуждается в установке.

Регулярные выражения
https://tproger.ru/translations/regular-expression-python/

Из модуля `re` нам понадобится только функция `findall()`, которая позволяет найти в строке все слова, удовлетворяющие шаблону. Мы находим в строке с названием фильма шаблон "(`DDDD`)" — четыре цифры, обёрнутых в скобки, что соответствует году выпуска фильма. Если такого шаблона не было найдено (год выпуска не указан), функция возвращает None (в таблице это будет помечено как пропуск).


In [17]:
#8.1 Создайте в таблице новый признак year_release, 
# который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?
year_release = ratings_movies['title'].apply(get_year_release)
ratings_movies['year_release'] = year_release
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


In [ ]:
# Посчитаем NaN значения по столбу год выпуска
ratings_movies['year_release'].isna().sum()

18

In [ ]:
# Фильм 1999 года с наименьшей средней зрительской оценкой
ratings_movies['year_release'].fillna(0, inplace=True)
ratings_movies['year_release'] = ratings_movies['year_release'].astype('int64')
mask = ratings_movies['year_release'] == 1999
ratings_movies[mask].groupby('title')['rating'].agg(['mean']).sort_values(by='mean')



,mean
title,
Bloodsport: The Dark Kumite (1999),0.5
Simon Sez (1999),1.0
Chill Factor (1999),1.0
"Source, The (1999)",1.0
Trippin' (1999),1.0
...,...
Trailer Park Boys (1999),5.0
Larry David: Curb Your Enthusiasm (1999),5.0
Sun Alley (Sonnenallee) (1999),5.0


In [ ]:
# 8.3 Какое сочетание жанров фильмов (genres), 
# выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
ratings_movies[ratings_movies['year_release'] == 2010].groupby(
    'genres')['rating'].agg(['mean']).sort_values(by='mean')

,mean
genres,
Action|Sci-Fi,1.000000
Action|Adventure|Horror,1.500000
Action|Drama|Fantasy,1.500000
Crime|Romance,1.500000
Adventure|Comedy|Fantasy,1.833333
...,...
Crime,4.750000
Comedy|Musical,5.000000
Animation|Drama|Fantasy|Mystery,5.000000


In [ ]:
# 8.4 Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) 
# жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.
ratings_movies.groupby(
    by='userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [ ]:
# 8.5 Найдите пользователя, который выставил наименьшее количество оценок, 
# но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
ratings_movies.groupby(
    by='userId')['rating'].agg(['count','mean']).sort_values(
        by=['count','mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [ ]:
# 8.6 Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший 
# средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок 
# (количество значений в столбце rating) больше 10.
temp_df = ratings_movies[ratings_movies['year_release'] == 2018].groupby(
    by='genres')['rating'].agg(['mean', 'count'])
temp_df[temp_df['count'] > 10].sort_values(by='mean', ascending=False)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [ ]:
# 8.7 Добавьте в таблицу новый признак year_rating — год выставления оценки. 
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма 
# от года выставления оценки и жанра. 
# Выберите верные варианты ответа, исходя из построенной таблицы:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'], dayfirst=False)
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
pivot_ratings = ratings_movies.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres'
)
pivot_ratings


genres,(no genres listed),Action,Action|Adventure,Action|Adventure|Animation,Action|Adventure|Animation|Children,Action|Adventure|Animation|Children|Comedy,Action|Adventure|Animation|Children|Comedy|Fantasy,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure|Animation|Children|Comedy|Romance,Action|Adventure|Animation|Children|Comedy|Sci-Fi,...,Romance|Thriller,Romance|War,Romance|Western,Sci-Fi,Sci-Fi|IMAX,Sci-Fi|Thriller,Sci-Fi|Thriller|IMAX,Thriller,War,Western
year_rating,,,,,,,,,,,,,,,,,,,,,
1996,NaN,2.730769,3.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.838095,NaN,3.117647
1997,NaN,3.538462,4.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.400000,NaN,3.923077,NaN,3.000000
1998,NaN,NaN,4.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.800000,NaN,NaN
1999,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000,NaN,NaN,NaN,NaN,4.000000,NaN,3.700000,4.5,4.000000
2000,NaN,2.588235,3.738462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000,NaN,3.0,3.416667,NaN,2.142857,NaN,3.087912,3.0,4.058824
2001,NaN,3.000000,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,2.500000,NaN,2.500000,NaN,3.477273,3.0,3.111111
2002,NaN,2.750000,4.304348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000,NaN,NaN,3.750000,NaN,3.600000,NaN,3.583333,3.5,3.000000
2003,NaN,3.833333,3.277778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.375,2.5,NaN,2.333333,NaN,3.142857,NaN,3.250000,3.0,4.000000
2004,NaN,2.700000,4.136364,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,...,3.000,3.0,3.5,2.125000,NaN,NaN,NaN,3.464286,3.0,3.800000


In [ ]:
#За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure 
# ни разу не получало среднюю оценку ниже 3.
pivot_ratings['Action|Adventure'] > 3

year_rating
1996    True
1997    True
1998    True
1999    True
2000    True
2001    True
2002    True
2003    True
2004    True
2005    True
2006    True
2007    True
2008    True
2009    True
2010    True
2011    True
2012    True
2013    True
2014    True
2015    True
2016    True
2017    True
2018    True
Name: Action|Adventure, dtype: bool

In [ ]:
# Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.
pivot_ratings[
    'Action|Adventure|Animation|Children|Comedy|IMAX'].max() == pivot_ratings[
        'Action|Adventure|Animation|Children|Comedy|IMAX'].loc[2010]

False

In [ ]:
# Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, 
# есть сочетание Animation|Children|Mystery.
pivot_ratings['Animation|Children|Mystery'].loc[2018] == 5

True

In [ ]:
#  Для жанра Comedy прослеживается тенденция падения рейтинга 
# с каждым годом (с 1996 по 2018).
pivot_ratings['Comedy'].diff().mean()
pivot_ratings['Comedy'].describe()

count    23.000000
mean      3.235808
std       0.191935
min       2.852668
25%       3.120681
50%       3.232877
75%       3.365154
max       3.606061
Name: Comedy, dtype: float64

In [18]:
orders = pd.read_csv('orders.csv', sep=';')

products = pd.read_csv('products.csv', sep=';')

orders

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


In [ ]:
# 8.8 Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?
orders['Product_ID'] = orders['ID товара']
orders_products = orders.merge(
    products,
    on='Product_ID',
    how='left'
)
orders_products[orders_products['Name'].isnull() == True]

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
17,01.01.2001 00:00:00,0,1,"Оплачен, формируется к отправке",Да,Нет,Нет,666,1,666,NaN,NaN,NaN


In [ ]:
# 8.9 На какой товар была произведена отмена?
orders_products[orders_products['Статус'] == 'Отменён']['Name']

6    Носки беговые Camino
Name: Name, dtype: object

In [ ]:
def get_income(arg):
    valid_status = ['Оплачен, формируется к отправке', 'Выполнен']
    if arg in valid_status:
        return 1
    else:
        return 0
    
orders_products['status'] = orders_products['Статус'].apply(get_income)
orders_products['income'] = orders_products['Количество'] * orders_products['Price'] * orders_products['status']
orders_products

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY,income,status
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103,"Носки Подарочные, муж",199.0,RUR,0.0,0
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86,"Носки Простые, муж",45.0,RUR,0.0,0
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104,"Носки Подарочные, жен",249.0,RUR,0.0,0
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104,"Носки Подарочные, жен",249.0,RUR,0.0,0
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104,"Носки Подарочные, жен",249.0,RUR,0.0,0
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103,"Носки Подарочные, муж",199.0,RUR,0.0,0
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124,Носки беговые Camino,999.0,RUR,0.0,0
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91,"Носки Честные, муж",50.0,RUR,0.0,0
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103,"Носки Подарочные, муж",199.0,RUR,597.0,1
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104,"Носки Подарочные, жен",249.0,RUR,747.0,1


In [ ]:
# 8.10 Какой покупатель принёс наибольшую суммарную прибыль магазину за указанный период?
# В ответ запишите идентификатор этого покупателя (ID Покупателя).

orders_products.groupby(
    by='ID Покупателя')['income'].agg(['count','sum']).sort_values(by='sum',ascending=False)

,count,sum
ID Покупателя,,
7,4,17096.0
5,4,13043.0
8,3,1344.0
1,2,0.0
9,3,0.0
10,1,0.0


[К содержанию](#0)